In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
16,It is so nice to see Bruce Willis come down of...,positive
52,This is an EXCELLENT example of early Bette Da...,positive
833,I have never understood the appeal of this sho...,negative
911,"Seeing this movie, as I just did for the first...",negative
881,"or: It's a bird ? It's a plane ? No, look... I...",negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

Error during text normalization: 
**********************************************************************
  Resource stopwords not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('stopwords')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/stopwords

  Searched in:
    - 'C:\\Users\\Kai/nltk_data'
    - 'c:\\Users\\Kai\\anaconda3\\envs\\atlas\\nltk_data'
    - 'c:\\Users\\Kai\\anaconda3\\envs\\atlas\\share\\nltk_data'
    - 'c:\\Users\\Kai\\anaconda3\\envs\\atlas\\lib\\nltk_data'
    - 'C:\\Users\\Kai\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Kai/nltk_data'
    - 'c:\\Users\\Kai\\anaconda3\\envs\\atlas\\nltk_data'
    - 'c:\\Users\\Kai\\anaconda3\\envs\\atlas\\share\\nltk_data'
    - 'c:\\Users\\Kai\\anaconda3\\envs\\atlas\\lib\\nltk_data'
    - 'C:\\Users\\Kai\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
16,it is so nice to see bruce willis come down of...,1
52,this is an excellent example of early bette da...,1
833,i have never understood the appeal of this sho...,0
911,"seeing this movie, as i just did for the first...",0
881,"or: it's a bird ? it's a plane ? no, look... i...",0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/kaidranzerr/MLOPS-capstone-project.mlflow')
dagshub.init(repo_owner='kaidranzerr', repo_name='MLOPS-capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as kaidranzerr

Initialized MLflow to track repo "kaidranzerr/MLOPS-capstone-project"

Repository kaidranzerr/MLOPS-capstone-project initialized!

2025/09/19 11:53:07 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/30ffd2aad3804543b0b0a8eea80c771b', creation_time=1758262989095, experiment_id='0', last_update_time=1758262989095, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib 

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")

        logging.info("Saving model locally...")
        joblib.dump(model, "log_reg_model.pkl")

        logging.info("Logging model to MLflow as artifact...")
        mlflow.log_artifact("log_reg_model.pkl", artifact_path="models")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-19 12:54:43,417 - INFO - Starting MLflow run...
2025-09-19 12:54:44,861 - INFO - Logging preprocessing parameters...
2025-09-19 12:54:46,313 - INFO - Initializing Logistic Regression model...
2025-09-19 12:54:46,314 - INFO - Fitting the model...
2025-09-19 12:54:46,442 - INFO - Model training complete.
2025-09-19 12:54:46,444 - INFO - Logging model parameters...
2025-09-19 12:54:46,914 - INFO - Making predictions...
2025-09-19 12:54:46,917 - INFO - Calculating evaluation metrics...
2025-09-19 12:54:46,940 - INFO - Logging evaluation metrics...
2025-09-19 12:54:52,641 - INFO - Saving model locally...
2025-09-19 12:54:52,648 - INFO - Logging model to MLflow as artifact...
2025-09-19 12:54:53,977 - INFO - Model training and logging completed in 9.12 seconds.
2025-09-19 12:54:53,978 - INFO - Accuracy: 0.656
2025-09-19 12:54:53,980 - INFO - Precision: 0.6451612903225806
2025-09-19 12:54:53,982 - INFO - Recall: 0.6557377049180327
2025-09-19 12:54:53,982 - INFO - F1 Score: 0.650406504

🏃 View run smiling-mole-620 at: https://dagshub.com/kaidranzerr/MLOPS-capstone-project.mlflow/#/experiments/0/runs/ddd5455704724b6fa93ab2cc8f28e36b
🧪 View experiment at: https://dagshub.com/kaidranzerr/MLOPS-capstone-project.mlflow/#/experiments/0
